# Vaje 4 - optimizacija hiperparametrov s knjižnico hyperopt

A.1 Hyperopt je nekoliko bolj zapletena optimizacija, zato bomo za začetek poskusili najti x, ki minimizira funkcijo
   optimalne f(x) = x^2. Kandidate za ta x bomo iskali na intervalu
   a) [-4, 4]
   b) [4, 8].
Oglejte si se druge moznosti zrebanja po prostoru: https://github.com/hyperopt/hyperopt/wiki/FMin (razdelek 2.1)

In [2]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from hyperopt import hp, tpe, rand, fmin, Trials, space_eval
from hyperopt import pyll, base
import numpy as np

prostorA = {"x": hp.uniform("x", -4, 4)}
prostorB = {"x": hp.uniform("x", 4, 8)}

def kriterijska_funkcija(parametri):
    x = parametri["x"]
    return x ** 2

trials = Trials()
best = fmin(fn = kriterijska_funkcija,
            space = prostorA,
            algo = tpe.suggest,
            max_evals = 100,
            trials = trials
)

best = space_eval(prostorA, best)
best_value = kriterijska_funkcija(best)

# vse vrednosti paramtrov in kriterijske funkcije, ki smo jih preizkusili
xs = [trial["misc"]["vals"]["x"][0] for trial in trials.trials]
ys = [trial["result"]["loss"] for trial in trials.trials]

print(best, best_value)

100%|██████████| 100/100 [00:00<00:00, 1566.30trial/s, best loss: 3.950613247985221e-05]
{'x': 0.006285390399955456} 3.950613247985221e-05


A.2 Pogosto je treba najti več kot en optimalni parameter. Minimiziraj funkcijo $f(x, y) = (x - y)^2 + (y - 1)^2$.
   Prostora ne preiskuj enakomerno, ampak z normalno porazdelitvijo.

In [6]:

def kriterijska_funkcija_2(parametri):
    x = parametri["x"]
    y = parametri["y"]
    return (x - y)*(x - y) + (y - 1)*(y - 1)

prostor_xy = {"x": hp.normal("x", 0, 4), "y" : hp.normal("y", 0, 4)}

trials = Trials()
best = fmin(fn = kriterijska_funkcija_2,
            space = prostor_xy,
            algo = tpe.suggest,
            max_evals = 200,
            trials = trials
)

best = space_eval(prostor_xy, best)
best_value = kriterijska_funkcija_2(best)
print(best, best_value)

100%|██████████| 200/200 [00:00<00:00, 1073.28trial/s, best loss: 0.014814026664388746]
{'x': 1.1549831041375125, 'y': 1.0400472909574776} 0.014814026664388746


A.3. Včasih moramo izvesti tudi diskretno izbiro, npr. ko izbiramo algoritem. Za spodnje podatke najdi najprimernejsi
   algoritem. Izbiraj med kNN, odlocitvenim drevesom in metodo podpornih vektorjev.

In [9]:
n_primerov = 1000
n_znacilk = 5

x = np.random.rand(n_primerov, n_znacilk)
y = np.dot(x, list(range(n_znacilk)))

povprecje = np.mean(y)
pozitivni = y >= povprecje
y[pozitivni] = 1
y[~pozitivni] = 0


In [11]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

prostor_algo = {"algo": hp.choice('algo', ['tree','knn','svm'])}

np.random.seed(0)
x_ucna, x_testna, y_ucna, y_testna = train_test_split(x, y, test_size=0.2)


def kriterijska_funkcija_algo(parametri):
    a = parametri["algo"]
    if a == 'tree':
        model = DecisionTreeClassifier()
    elif a == 'knn':
        model = KNeighborsClassifier()
    elif a == 'svm':
        model = SVC()

    model.fit(x_ucna, y_ucna)
    y_napoved = model.predict(x_testna)
    return 1 - accuracy_score(y_testna, y_napoved)

trials = Trials()
best = fmin(
            fn = kriterijska_funkcija_algo,
            space = prostor_algo,
            algo = tpe.suggest,
            max_evals = 100,
            trials = trials
            )

best = space_eval(prostor_algo, best)
best_value = kriterijska_funkcija_algo(best)

xs = [trial["misc"]["vals"] for trial in trials]
acc = [trial["result"]["loss"] for trial in trials]
print(xs)
print(acc)
print(best, best_value)


100%|██████████| 100/100 [00:00<00:00, 234.84trial/s, best loss: 0.03500000000000003]
[{'algo': [1]}, {'algo': [2]}, {'algo': [1]}, {'algo': [1]}, {'algo': [0]}, {'algo': [2]}, {'algo': [1]}, {'algo': [0]}, {'algo': [0]}, {'algo': [2]}, {'algo': [2]}, {'algo': [0]}, {'algo': [0]}, {'algo': [1]}, {'algo': [0]}, {'algo': [2]}, {'algo': [2]}, {'algo': [0]}, {'algo': [0]}, {'algo': [1]}, {'algo': [2]}, {'algo': [2]}, {'algo': [2]}, {'algo': [2]}, {'algo': [2]}, {'algo': [2]}, {'algo': [2]}, {'algo': [2]}, {'algo': [2]}, {'algo': [2]}, {'algo': [2]}, {'algo': [2]}, {'algo': [1]}, {'algo': [2]}, {'algo': [2]}, {'algo': [1]}, {'algo': [2]}, {'algo': [2]}, {'algo': [1]}, {'algo': [2]}, {'algo': [2]}, {'algo': [1]}, {'algo': [2]}, {'algo': [0]}, {'algo': [2]}, {'algo': [0]}, {'algo': [2]}, {'algo': [1]}, {'algo': [2]}, {'algo': [0]}, {'algo': [2]}, {'algo': [1]}, {'algo': [0]}, {'algo': [2]}, {'algo': [2]}, {'algo': [2]}, {'algo': [0]}, {'algo': [2]}, {'algo': [1]}, {'algo': [2]}, {'algo': [2]}

A.4 Poleg samega algoritma želimo iskati tudi optimalne vrednosti hiperparametrov. Seveda so hiperparametri odvisni od
   samega algoritma, zato je preiskovani prostor treba definirati gnezdeno, kot kaze primer.
   Razsiri preiskovani prostor tako, da dodas se kaksen parameter. Priporocamo, da jih poimenujes tako, kot so
   poimenovani pripadajoci argumenti v konstruktorju za dani model (npr. eden od parametrov za drevo je max_depth,
   ki pove, katera je najvecja dovoljena globina drevesa)

   Razsiri gnezdeni_prostor_spodaj.

In [12]:
gnezdeni_prostor = {
    "algo": hp.choice('algo', [
        {
            'ime': 'tree',
            'max_depth': hp.choice('max_depth', [2, 4, 8, 16, 32]),
            'min_samples_split' : hp.choice('min_samples_split', [2, 4, 8, 16, 32]),
            'min_samples_leaf' : hp.choice('min_samples_leaf', [1, 2, 3])

        },
        {
            'ime': 'knn',
            'n_neighbors' : hp.choice('n_neighbors', [5, 10, 15, 20, 25, 30])
        },
        {
            'ime': 'svm',
            'C': hp.lognormal('C', 0, 1),
            'kernel': hp.choice('kernel', [
                {
                    'tip': 'linear'  # linearno
                },
                {
                    'tip': 'rbf',  # radialno
                    'gamma': hp.lognormal('gamma', 0, 1)
                },
                {
                    'tip': 'poly',  # polinomsko
                    'degree': hp.choice("degree", [1, 2, 3])
                }
            ]),
        },
    ])
}

A.5. Kako dobre so prednastavljene vrednosti? Primerjaj rezultate, ki jih dobis s preiskovanjem prostorov
   - prostor_algo (tu bodo pri algoritmih vedno uporabljene prednastavljene vrednosti)
   - gnezdeni_prostor

   Kateri algoritem (in katera konfiguracija parametrov) je najboljsa za dane podatke?
   Preizkusi na vseh treh moznih podatkovjih.

In [18]:
def izracunaj_y(xs, tip_podatkov):
    if tip_podatkov not in [1, 2, 3]:
        raise ValueError("Tip podatkov mora biti element [1, 2, 3].")
    if tip_podatkov == 1:
        r = list(range(xs.shape[1]))
        cs = np.array(r).reshape((-1, 1))
        ys = np.dot(xs, cs).reshape((-1,))
        poz = ys >= np.mean(ys)
        ys = np.array(["a" if p else "b" for p in poz])
    elif tip_podatkov == 2:
        group1 = xs[:, 1] > 0.5
        group2 = (xs[:, 1] <= 0.5) & (xs[:, 2] > 0.2)
        group3 = (xs[:, 1] <= 0.5) & (xs[:, 2] <= 0.2)
        ys = np.zeros(xs.shape[0], dtype=str)
        ys[group1] = "a"
        ys[group2] = "b"
        ys[group3] = "c"
    else:
        circle = np.sum(np.square(xs), axis=1) > 0.4
        ys = np.zeros(xs.shape[0], dtype=str)
        ys[circle] = "a"
        ys[~circle] = "b"
    return ys


def pripravi_podatke(n_ucna, n_test, n_znacilke, p_sum, tip_podatkov):
    x = np.random.rand(n_ucna + n_test, n_znacilke)
    y = izracunaj_y(x, tip_podatkov)
    vrednosti = list(set(y))
    for i in range(len(y)):
        # zamenjamo vrednost
        if np.random.rand(1) < p_sum:
            i0 = vrednosti.index(y[i])
            y[i] = vrednosti[(i0 + 1) % len(vrednosti)]
    x_ucna = x[:n_ucna]
    y_ucna = y[:n_ucna]
    x_test = x[n_ucna:]
    y_test = y[n_ucna:]
    return x_ucna, x_test, y_ucna, y_test


#x_ucna, x_testna, y_ucna, y_testna = pripravi_podatke(1000, 200, 5, 0.1, 2)

def kriterijska_funkcija_privzete(parametri):
    a = parametri["algo"]
    ime_algoritma = a["ime"]
    if ime_algoritma == 'tree':
        model = DecisionTreeClassifier()
    elif ime_algoritma == 'knn':
        model = KNeighborsClassifier()
    elif ime_algoritma == 'svm':
        model = SVC()

    model.fit(x_ucna, y_ucna)
    y_napoved = model.predict(x_testna)
    return 1 - accuracy_score(y_testna, y_napoved)
    # DOPOLNI

def kriterijska_funkcija_razsirjeno(parametri):
    a = parametri["algo"]
    ime_algoritma = a["ime"]
    if ime_algoritma == "knn":
        model = KNeighborsClassifier(n_neighbors=a["n_neighbors"])

    elif ime_algoritma == "tree":
        model = DecisionTreeClassifier(max_depth=a['max_depth'], 
                                       min_samples_split=a['min_samples_split'], 
                                       min_samples_leaf=a['min_samples_leaf'])

    elif ime_algoritma == "svm":
        kernel = a["kernel"]["tip"]
        C = a['C']
        # gamma in degree moramo definirati v vseh treh primerih: tam, kjer nista vazni, ju damo na 1
        neumna_vrednost = 1
        if kernel == "rbf":
            gamma = a["kernel"]["gamma"]            
            degree = neumna_vrednost
        elif kernel == "linear":
            gamma = neumna_vrednost
            degree = neumna_vrednost
        elif kernel == "poly":
            gamma = neumna_vrednost
            degree = a["kernel"]["degree"]
        model = SVC(kernel=kernel, gamma=gamma, C=C, degree=degree)
    else:
        raise ValueError("Napacne nastavitve!")
    
    model.fit(x_ucna, y_ucna)
    y_hat = model.predict(x_testna)
    return 1 - accuracy_score(y_testna, y_hat)




for i in [1, 2, 3]:
    x_ucna, x_testna, y_ucna, y_testna = pripravi_podatke(1000, 200, 5, 0.2, i)
    trials1 = Trials()
    best1 = fmin(
                fn = kriterijska_funkcija_privzete,
                space = gnezdeni_prostor,
                algo = tpe.suggest,
                max_evals = 100,
                trials = trials1
                )

    best1 = space_eval(gnezdeni_prostor, best1)
    best_value1 = kriterijska_funkcija_privzete(best1)
    trials2 = Trials()
    best2 = fmin(
                fn = kriterijska_funkcija_razsirjeno,
                space = gnezdeni_prostor,
                algo = tpe.suggest,
                max_evals = 100,
                trials = trials2
                )

    best2 = space_eval(gnezdeni_prostor, best2)
    best_value2 = kriterijska_funkcija_privzete(best2)
    print(f"On data {i} best algo with default params {best1}, best value {best_value1}")
    print(f"On data {i} best algo with best params {best2}, best value {best_value2}")




100%|██████████| 100/100 [00:01<00:00, 54.79trial/s, best loss: 0.19999999999999996]
On data 1 best algo with default params {'algo': {'C': 0.34792601247366106, 'ime': 'svm', 'kernel': {'tip': 'linear'}}}, best value 0.22999999999999998
On data 1 best algo with best params {'algo': {'C': 1.7059766336771969, 'ime': 'svm', 'kernel': {'degree': 2, 'tip': 'poly'}}}, best value 0.22999999999999998
100%|██████████| 100/100 [00:00<00:00, 157.56trial/s, best loss: 0.19499999999999995]
On data 2 best algo with default params {'algo': {'ime': 'tree', 'max_depth': 4, 'min_samples_leaf': 2, 'min_samples_split': 32}}, best value 0.31000000000000005
On data 2 best algo with best params {'algo': {'ime': 'tree', 'max_depth': 2, 'min_samples_leaf': 3, 'min_samples_split': 4}}, best value 0.32499999999999996
100%|██████████| 100/100 [00:47<00:00,  2.11trial/s, best loss: 0.20999999999999996]
On data 3 best algo with default params {'algo': {'C': 3.1262797327036878, 'ime': 'svm', 'kernel': {'degree': 1, 

hyperopt-sklearn ... podobno kot hyperopt samo da je od sklearna
TPOT ... podobno samo rabijo veliko casa, ampak sam dela vse
AutoML ... rabijo veliko casa, ampak sam dela vse